### 실제 모델

### dataset 생성

In [338]:
# train을 위한 dataset 생성

# 범주형 칼럼 int 형으로 변경
before.loc[before["AGE"]=="1020s","AGE"]=0
before.loc[before["AGE"]=="3060s","AGE"]=1
before.loc[before["FLC"]=="family","FLC"]=0
before.loc[before["FLC"]=="single","FLC"]=1

after.loc[after["AGE"]=="1020s","AGE"]=0
after.loc[after["AGE"]=="3060s","AGE"]=1
after.loc[after["FLC"]=="family","FLC"]=0
after.loc[after["FLC"]=="single","FLC"]=1

# home과 out을 나눔
before_home = before[before["CARD_SIDO_NM"]==before["HOM_SIDO_NM"]]
before_out = before[before["CARD_SIDO_NM"]!=before["HOM_SIDO_NM"]]

after_home = after[after["CARD_SIDO_NM"]==after["HOM_SIDO_NM"]]
after_out = after[after["CARD_SIDO_NM"]!=after["HOM_SIDO_NM"]]

# 기준이 되는 칼럼들로 AMT 묶기
columns = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'AGE','year','FLC','month']
before_home = before_home.groupby(columns).sum().reset_index(drop=False)
before_out = before_out.groupby(columns).sum().reset_index(drop=False)
after_home = after_home.groupby(columns).sum().reset_index(drop=False)
after_out = after_out.groupby(columns).sum().reset_index(drop=False)

# 차이값 계산
columns = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'AGE','FLC','month']
home_train = after_home.merge(before_home, on = columns)
home_train["corona_effect"] = home_train["AMT_y"]-home_train["AMT_x"]
out_train = after_out.merge(before_out, on = columns)
out_train["corona_effect"] = out_train["AMT_y"]-out_train["AMT_x"]

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [491]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn import metrics
import math
from sklearn.model_selection import train_test_split

select_columns = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'AGE', 'FLC',"month",
       'out_home_x', 'diff_x', '검색량(=100)_x', 'confirmed_x', '소비자심리지수_x']
# 데이터 준비
home_X = home_train.loc[:,select_columns]
home_Y = home_train["corona_effect"]

out_X = out_train.loc[:,select_columns]
out_Y = out_train["corona_effect"]
"""
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(home_X["CARD_SIDO_NM"])
home_X["CARD_SIDO_NM"]=le.transform(home_X["CARD_SIDO_NM"])

le.fit(home_X["STD_CLSS_NM"])
home_X["STD_CLSS_NM"]=le.transform(home_X["STD_CLSS_NM"])

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(out_X["CARD_SIDO_NM"])
out_X["CARD_SIDO_NM"]=le.transform(out_X["CARD_SIDO_NM"])

le.fit(out_X["STD_CLSS_NM"])
out_X["STD_CLSS_NM"]=le.transform(out_X["STD_CLSS_NM"])
"""
onehot_columns = ['CARD_SIDO_NM', 'STD_CLSS_NM',"month"]
add_home = pd.get_dummies(home_X.loc[:,onehot_columns])
home_X = home_X.drop(onehot_columns, axis=1)

add_out = pd.get_dummies(out_X.loc[:,onehot_columns])
out_X = out_X.drop(onehot_columns, axis=1)

home_X = pd.concat([home_X, add_home], axis=1)
out_X = pd.concat([out_X, add_out], axis=1)


In [492]:

from sklearn.preprocessing import MinMaxScaler
le=MinMaxScaler().fit(home_X)
home_X = le.transform(home_X)

le=MinMaxScaler().fit(out_X)
out_X = le.transform(out_X)

### model 생성

In [ ]:
# 모델을 평가할 평가 지표 생성
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score

def get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion=confusion_matrix(y_test,pred)
    accuracy = accuracy_score(y_test,pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    
    # ROC-AUC 추가
    roc_auc = roc_auc_score(y_test, pred_proba)
    
def rmsle(estimator, X, y0):
    pred = estimator.predict(X)
    squared_error = (np.log((y0+1)/(pred+1)))** 2
    rmsle = np.sqrt(np.mean(squared_error))
    if math.isnan(rmsle):
        print("this is a nan")

    return -rmsle

In [455]:
import xgboost as xgb
from xgboost import plot_importance

# XGB 부스터에 맞게 조정
# Dmatrix : numpy 입력 파라미터를 받아서 만들어지는 XGBoost만의 전용 데이터셋
# pandas의 데이터셋이라면 numpy의 데이터셋으로 변경해주는 것이 필요함
dhome = xgb.DMatrix(data=home_X, label=home_Y)
dout = xgb.DMatrix(data=out_X, label=out_Y)
wlist_home = [(dhome,"train"),(dhome,"eval")]
wlist_out = [(dout,"train"),(dout,"eval")]

param_grid = {"max_depth":7, "eta":0.1,"eval_metric":"logloss"}
num_rounds = 3000

# 하이퍼 파라미터와 early stopping 파라미터를 train()함수의 파라미터로 전달
xgb_home_model = xgb.train(dtrain=dhome, num_boost_round=num_rounds, params=param_grid,
                     early_stopping_rounds=400, evals=wlist_home)

xgb_out_model = xgb.train(dtrain=dout, num_boost_round=num_rounds, params=param_grid,
                     early_stopping_rounds=400, evals=wlist_out)

# eval 파라미터에 학습 데이터 셋과 eval 데이터 셋을 명시해주면 eval 데이터셋에 대해
# 평가를 수행하면서 조기 중단 적용 가능

# 예측
home_predicted = xgb_home_model.predict(dhome)
out_predicted = xgb_out_model.predict(dout)

[0]	train-logloss:-9.64991e+09	eval-logloss:-9.64991e+09
Multiple eval metrics have been passed: 'eval-logloss' will be used for early stopping.

Will train until eval-logloss hasn't improved in 400 rounds.
[1]	train-logloss:-9.71626e+09	eval-logloss:-9.71626e+09
[2]	train-logloss:-9.75812e+09	eval-logloss:-9.75812e+09
[3]	train-logloss:-9.75994e+09	eval-logloss:-9.75994e+09
[4]	train-logloss:-9.73308e+09	eval-logloss:-9.73308e+09
[5]	train-logloss:-9.71144e+09	eval-logloss:-9.71144e+09
[6]	train-logloss:-9.65004e+09	eval-logloss:-9.65004e+09
[7]	train-logloss:-9.62016e+09	eval-logloss:-9.62016e+09
[8]	train-logloss:-9.46707e+09	eval-logloss:-9.46707e+09
[9]	train-logloss:-9.4721e+09	eval-logloss:-9.4721e+09
[10]	train-logloss:-9.50838e+09	eval-logloss:-9.50838e+09
[11]	train-logloss:-9.6725e+09	eval-logloss:-9.6725e+09
[12]	train-logloss:-9.71092e+09	eval-logloss:-9.71092e+09
[13]	train-logloss:-9.73444e+09	eval-logloss:-9.73444e+09
[14]	train-logloss:-9.69355e+09	eval-logloss:-9.6935

[139]	train-logloss:-1.01804e+10	eval-logloss:-1.01804e+10
[140]	train-logloss:-1.0177e+10	eval-logloss:-1.0177e+10
[141]	train-logloss:-1.01785e+10	eval-logloss:-1.01785e+10
[142]	train-logloss:-1.01814e+10	eval-logloss:-1.01814e+10
[143]	train-logloss:-1.01804e+10	eval-logloss:-1.01804e+10
[144]	train-logloss:-1.01797e+10	eval-logloss:-1.01797e+10
[145]	train-logloss:-1.01863e+10	eval-logloss:-1.01863e+10
[146]	train-logloss:-1.01832e+10	eval-logloss:-1.01832e+10
[147]	train-logloss:-1.01862e+10	eval-logloss:-1.01862e+10
[148]	train-logloss:-1.01875e+10	eval-logloss:-1.01875e+10
[149]	train-logloss:-1.01872e+10	eval-logloss:-1.01872e+10
[150]	train-logloss:-1.01851e+10	eval-logloss:-1.01851e+10
[151]	train-logloss:-1.01867e+10	eval-logloss:-1.01867e+10
[152]	train-logloss:-1.01878e+10	eval-logloss:-1.01878e+10
[153]	train-logloss:-1.01882e+10	eval-logloss:-1.01882e+10
[154]	train-logloss:-1.01906e+10	eval-logloss:-1.01906e+10
[155]	train-logloss:-1.01903e+10	eval-logloss:-1.01903e+10

[279]	train-logloss:-1.02601e+10	eval-logloss:-1.02601e+10
[280]	train-logloss:-1.02592e+10	eval-logloss:-1.02592e+10
[281]	train-logloss:-1.02584e+10	eval-logloss:-1.02584e+10
[282]	train-logloss:-1.02563e+10	eval-logloss:-1.02563e+10
[283]	train-logloss:-1.02542e+10	eval-logloss:-1.02542e+10
[284]	train-logloss:-1.0256e+10	eval-logloss:-1.0256e+10
[285]	train-logloss:-1.02562e+10	eval-logloss:-1.02562e+10
[286]	train-logloss:-1.02579e+10	eval-logloss:-1.02579e+10
[287]	train-logloss:-1.0259e+10	eval-logloss:-1.0259e+10
[288]	train-logloss:-1.02588e+10	eval-logloss:-1.02588e+10
[289]	train-logloss:-1.02607e+10	eval-logloss:-1.02607e+10
[290]	train-logloss:-1.02611e+10	eval-logloss:-1.02611e+10
[291]	train-logloss:-1.02596e+10	eval-logloss:-1.02596e+10
[292]	train-logloss:-1.02608e+10	eval-logloss:-1.02608e+10
[293]	train-logloss:-1.02669e+10	eval-logloss:-1.02669e+10
[294]	train-logloss:-1.02649e+10	eval-logloss:-1.02649e+10
[295]	train-logloss:-1.02641e+10	eval-logloss:-1.02641e+10
[

[419]	train-logloss:-1.02938e+10	eval-logloss:-1.02938e+10
[420]	train-logloss:-1.02934e+10	eval-logloss:-1.02934e+10
[421]	train-logloss:-1.02936e+10	eval-logloss:-1.02936e+10
[422]	train-logloss:-1.02943e+10	eval-logloss:-1.02943e+10
[423]	train-logloss:-1.02928e+10	eval-logloss:-1.02928e+10
[424]	train-logloss:-1.02946e+10	eval-logloss:-1.02946e+10
[425]	train-logloss:-1.0295e+10	eval-logloss:-1.0295e+10
[426]	train-logloss:-1.02949e+10	eval-logloss:-1.02949e+10
[427]	train-logloss:-1.02951e+10	eval-logloss:-1.02951e+10
[428]	train-logloss:-1.02945e+10	eval-logloss:-1.02945e+10
[429]	train-logloss:-1.02939e+10	eval-logloss:-1.02939e+10
[430]	train-logloss:-1.02943e+10	eval-logloss:-1.02943e+10
[431]	train-logloss:-1.02922e+10	eval-logloss:-1.02922e+10
[432]	train-logloss:-1.0294e+10	eval-logloss:-1.0294e+10
[433]	train-logloss:-1.02947e+10	eval-logloss:-1.02947e+10
[434]	train-logloss:-1.02938e+10	eval-logloss:-1.02938e+10
[435]	train-logloss:-1.02934e+10	eval-logloss:-1.02934e+10
[

[559]	train-logloss:-1.03149e+10	eval-logloss:-1.03149e+10
[560]	train-logloss:-1.03149e+10	eval-logloss:-1.03149e+10
[561]	train-logloss:-1.03146e+10	eval-logloss:-1.03146e+10
[562]	train-logloss:-1.03142e+10	eval-logloss:-1.03142e+10
[563]	train-logloss:-1.03147e+10	eval-logloss:-1.03147e+10
[564]	train-logloss:-1.03145e+10	eval-logloss:-1.03145e+10
[565]	train-logloss:-1.03159e+10	eval-logloss:-1.03159e+10
[566]	train-logloss:-1.03155e+10	eval-logloss:-1.03155e+10
[567]	train-logloss:-1.03154e+10	eval-logloss:-1.03154e+10
[568]	train-logloss:-1.03145e+10	eval-logloss:-1.03145e+10
[569]	train-logloss:-1.03147e+10	eval-logloss:-1.03147e+10
[570]	train-logloss:-1.03159e+10	eval-logloss:-1.03159e+10
[571]	train-logloss:-1.03157e+10	eval-logloss:-1.03157e+10
[572]	train-logloss:-1.03155e+10	eval-logloss:-1.03155e+10
[573]	train-logloss:-1.03156e+10	eval-logloss:-1.03156e+10
[574]	train-logloss:-1.03149e+10	eval-logloss:-1.03149e+10
[575]	train-logloss:-1.03149e+10	eval-logloss:-1.03149e+

...

[2398]	train-logloss:-5.19178e+09	eval-logloss:-5.19178e+09
[2399]	train-logloss:-5.19178e+09	eval-logloss:-5.19178e+09
[2400]	train-logloss:-5.19179e+09	eval-logloss:-5.19179e+09
[2401]	train-logloss:-5.19178e+09	eval-logloss:-5.19178e+09
[2402]	train-logloss:-5.19178e+09	eval-logloss:-5.19178e+09
[2403]	train-logloss:-5.19178e+09	eval-logloss:-5.19178e+09
[2404]	train-logloss:-5.19179e+09	eval-logloss:-5.19179e+09
[2405]	train-logloss:-5.19179e+09	eval-logloss:-5.19179e+09
[2406]	train-logloss:-5.19179e+09	eval-logloss:-5.19179e+09
[2407]	train-logloss:-5.1918e+09	eval-logloss:-5.1918e+09
[2408]	train-logloss:-5.19179e+09	eval-logloss:-5.19179e+09
[2409]	train-logloss:-5.19179e+09	eval-logloss:-5.19179e+09
[2410]	train-logloss:-5.1918e+09	eval-logloss:-5.1918e+09
[2411]	train-logloss:-5.19179e+09	eval-logloss:-5.19179e+09
[2412]	train-logloss:-5.1918e+09	eval-logloss:-5.1918e+09
[2413]	train-logloss:-5.19179e+09	eval-logloss:-5.19179e+09
[2414]	train-logloss:-5.19179e+09	eval-logloss

[2536]	train-logloss:-5.19193e+09	eval-logloss:-5.19193e+09
[2537]	train-logloss:-5.19193e+09	eval-logloss:-5.19193e+09
[2538]	train-logloss:-5.19193e+09	eval-logloss:-5.19193e+09
[2539]	train-logloss:-5.19193e+09	eval-logloss:-5.19193e+09
[2540]	train-logloss:-5.19193e+09	eval-logloss:-5.19193e+09
[2541]	train-logloss:-5.19194e+09	eval-logloss:-5.19194e+09
[2542]	train-logloss:-5.19194e+09	eval-logloss:-5.19194e+09
[2543]	train-logloss:-5.19194e+09	eval-logloss:-5.19194e+09
[2544]	train-logloss:-5.19194e+09	eval-logloss:-5.19194e+09
[2545]	train-logloss:-5.19195e+09	eval-logloss:-5.19195e+09
[2546]	train-logloss:-5.19195e+09	eval-logloss:-5.19195e+09
[2547]	train-logloss:-5.19195e+09	eval-logloss:-5.19195e+09
[2548]	train-logloss:-5.19195e+09	eval-logloss:-5.19195e+09
[2549]	train-logloss:-5.19195e+09	eval-logloss:-5.19195e+09
[2550]	train-logloss:-5.19195e+09	eval-logloss:-5.19195e+09
[2551]	train-logloss:-5.19195e+09	eval-logloss:-5.19195e+09
[2552]	train-logloss:-5.19195e+09	eval-l

[2673]	train-logloss:-5.192e+09	eval-logloss:-5.192e+09
[2674]	train-logloss:-5.192e+09	eval-logloss:-5.192e+09
[2675]	train-logloss:-5.192e+09	eval-logloss:-5.192e+09
[2676]	train-logloss:-5.192e+09	eval-logloss:-5.192e+09
[2677]	train-logloss:-5.192e+09	eval-logloss:-5.192e+09
[2678]	train-logloss:-5.192e+09	eval-logloss:-5.192e+09
[2679]	train-logloss:-5.192e+09	eval-logloss:-5.192e+09
[2680]	train-logloss:-5.192e+09	eval-logloss:-5.192e+09
[2681]	train-logloss:-5.192e+09	eval-logloss:-5.192e+09
[2682]	train-logloss:-5.192e+09	eval-logloss:-5.192e+09
[2683]	train-logloss:-5.192e+09	eval-logloss:-5.192e+09
[2684]	train-logloss:-5.192e+09	eval-logloss:-5.192e+09
[2685]	train-logloss:-5.192e+09	eval-logloss:-5.192e+09
[2686]	train-logloss:-5.192e+09	eval-logloss:-5.192e+09
[2687]	train-logloss:-5.192e+09	eval-logloss:-5.192e+09
[2688]	train-logloss:-5.192e+09	eval-logloss:-5.192e+09
[2689]	train-logloss:-5.192e+09	eval-logloss:-5.192e+09
[2690]	train-logloss:-5.192e+09	eval-logloss:-5.

[2812]	train-logloss:-5.19211e+09	eval-logloss:-5.19211e+09
[2813]	train-logloss:-5.19212e+09	eval-logloss:-5.19212e+09
[2814]	train-logloss:-5.19212e+09	eval-logloss:-5.19212e+09
[2815]	train-logloss:-5.19212e+09	eval-logloss:-5.19212e+09
[2816]	train-logloss:-5.19212e+09	eval-logloss:-5.19212e+09
[2817]	train-logloss:-5.19212e+09	eval-logloss:-5.19212e+09
[2818]	train-logloss:-5.19212e+09	eval-logloss:-5.19212e+09
[2819]	train-logloss:-5.19212e+09	eval-logloss:-5.19212e+09
[2820]	train-logloss:-5.19212e+09	eval-logloss:-5.19212e+09
[2821]	train-logloss:-5.19212e+09	eval-logloss:-5.19212e+09
[2822]	train-logloss:-5.19212e+09	eval-logloss:-5.19212e+09
[2823]	train-logloss:-5.19212e+09	eval-logloss:-5.19212e+09
[2824]	train-logloss:-5.19212e+09	eval-logloss:-5.19212e+09
[2825]	train-logloss:-5.19212e+09	eval-logloss:-5.19212e+09
[2826]	train-logloss:-5.19212e+09	eval-logloss:-5.19212e+09
[2827]	train-logloss:-5.19212e+09	eval-logloss:-5.19212e+09
[2828]	train-logloss:-5.19212e+09	eval-l

[2949]	train-logloss:-5.19218e+09	eval-logloss:-5.19218e+09
[2950]	train-logloss:-5.19218e+09	eval-logloss:-5.19218e+09
[2951]	train-logloss:-5.19218e+09	eval-logloss:-5.19218e+09
[2952]	train-logloss:-5.19219e+09	eval-logloss:-5.19219e+09
[2953]	train-logloss:-5.19219e+09	eval-logloss:-5.19219e+09
[2954]	train-logloss:-5.19219e+09	eval-logloss:-5.19219e+09
[2955]	train-logloss:-5.19219e+09	eval-logloss:-5.19219e+09
[2956]	train-logloss:-5.19219e+09	eval-logloss:-5.19219e+09
[2957]	train-logloss:-5.19219e+09	eval-logloss:-5.19219e+09
[2958]	train-logloss:-5.19219e+09	eval-logloss:-5.19219e+09
[2959]	train-logloss:-5.19219e+09	eval-logloss:-5.19219e+09
[2960]	train-logloss:-5.19219e+09	eval-logloss:-5.19219e+09
[2961]	train-logloss:-5.19219e+09	eval-logloss:-5.19219e+09
[2962]	train-logloss:-5.19219e+09	eval-logloss:-5.19219e+09
[2963]	train-logloss:-5.19219e+09	eval-logloss:-5.19219e+09
[2964]	train-logloss:-5.1922e+09	eval-logloss:-5.1922e+09
[2965]	train-logloss:-5.1922e+09	eval-logl

In [456]:
def predict_rmsle(pred,y):
    squared_error = (np.log((y+1)/(pred+1)))** 2
    rmsle = np.sqrt(np.mean(squared_error))
    if math.isnan(rmsle):
        print("this is a nan")

    return rmsle

print(predict_rmsle(home_predicted, home_Y))
print(predict_rmsle(out_predicted, out_Y))

0.8867879841953322
0.4064936224590234


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\series.py:679: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [495]:
import xgboost as xgb
from xgboost import plot_importance
from sklearn.model_selection import train_test_split
train_home_X, test_home_X, train_home_Y, test_home_Y = train_test_split(home_X, home_Y, test_size=0.2, random_state=0)
train_out_X, test_out_X, train_out_Y, test_out_Y= train_test_split(out_X, out_Y, test_size=0.2, random_state=0)

# XGB 부스터에 맞게 조정
# Dmatrix : numpy 입력 파라미터를 받아서 만들어지는 XGBoost만의 전용 데이터셋
# pandas의 데이터셋이라면 numpy의 데이터셋으로 변경해주는 것이 필요함
train_dhome = xgb.DMatrix(data=train_home_X, label=train_home_Y)
train_dout = xgb.DMatrix(data=train_out_X, label=train_out_Y)

test_dhome = xgb.DMatrix(data=test_home_X, label=test_home_Y)
test_dout = xgb.DMatrix(data=test_out_X, label=test_out_Y)

wlist_home = [(train_dhome,"train"),(test_dhome,"eval")]
wlist_out = [(train_dout,"train"),(test_dout,"eval")]

param_grid = {"max_depth":15, "eta":0.1,"eval_metric":"logloss"}
num_rounds = 1000

# 하이퍼 파라미터와 early stopping 파라미터를 train()함수의 파라미터로 전달
xgb_home_model = xgb.train(dtrain=train_dhome, num_boost_round=num_rounds, params=param_grid,
                     early_stopping_rounds=400,evals=wlist_home)

xgb_out_model = xgb.train(dtrain=train_dout, num_boost_round=num_rounds, params=param_grid,
                     early_stopping_rounds=400, evals=wlist_out)

# eval 파라미터에 학습 데이터 셋과 eval 데이터 셋을 명시해주면 eval 데이터셋에 대해
# 평가를 수행하면서 조기 중단 적용 가능

# 예측
home_predicted = xgb_home_model.predict(test_dhome)
out_predicted = xgb_out_model.predict(test_dout)

[0]	train-logloss:-1.03977e+10	eval-logloss:-9.36642e+09
Multiple eval metrics have been passed: 'eval-logloss' will be used for early stopping.

Will train until eval-logloss hasn't improved in 400 rounds.
[1]	train-logloss:-1.04058e+10	eval-logloss:-9.41709e+09
[2]	train-logloss:-1.04008e+10	eval-logloss:-9.33903e+09
[3]	train-logloss:-1.04242e+10	eval-logloss:-9.37763e+09
[4]	train-logloss:-1.04009e+10	eval-logloss:-9.37323e+09
[5]	train-logloss:-1.0433e+10	eval-logloss:-9.35189e+09
[6]	train-logloss:-1.04359e+10	eval-logloss:-9.34871e+09
[7]	train-logloss:-1.04407e+10	eval-logloss:-9.36078e+09
[8]	train-logloss:-1.04436e+10	eval-logloss:-9.41612e+09
[9]	train-logloss:-1.04472e+10	eval-logloss:-9.42091e+09
[10]	train-logloss:-1.04486e+10	eval-logloss:-9.43239e+09
[11]	train-logloss:-1.04431e+10	eval-logloss:-9.428e+09
[12]	train-logloss:-1.04477e+10	eval-logloss:-9.43012e+09
[13]	train-logloss:-1.04474e+10	eval-logloss:-9.47038e+09
[14]	train-logloss:-1.04491e+10	eval-logloss:-9.485

[139]	train-logloss:-1.05196e+10	eval-logloss:-9.4705e+09
[140]	train-logloss:-1.05199e+10	eval-logloss:-9.47597e+09
[141]	train-logloss:-1.05201e+10	eval-logloss:-9.47653e+09
[142]	train-logloss:-1.05207e+10	eval-logloss:-9.47797e+09
[143]	train-logloss:-1.05214e+10	eval-logloss:-9.47604e+09
[144]	train-logloss:-1.05216e+10	eval-logloss:-9.47578e+09
[145]	train-logloss:-1.05216e+10	eval-logloss:-9.47784e+09
[146]	train-logloss:-1.05216e+10	eval-logloss:-9.47539e+09
[147]	train-logloss:-1.05216e+10	eval-logloss:-9.47538e+09
[148]	train-logloss:-1.05217e+10	eval-logloss:-9.47705e+09
[149]	train-logloss:-1.05217e+10	eval-logloss:-9.47732e+09
[150]	train-logloss:-1.0522e+10	eval-logloss:-9.47733e+09
[151]	train-logloss:-1.0522e+10	eval-logloss:-9.47953e+09
[152]	train-logloss:-1.0522e+10	eval-logloss:-9.4815e+09
[153]	train-logloss:-1.05222e+10	eval-logloss:-9.48105e+09
[154]	train-logloss:-1.05223e+10	eval-logloss:-9.47858e+09
[155]	train-logloss:-1.05222e+10	eval-logloss:-9.47861e+09
[1

[279]	train-logloss:-1.0537e+10	eval-logloss:-9.47762e+09
[280]	train-logloss:-1.0537e+10	eval-logloss:-9.47762e+09
[281]	train-logloss:-1.0537e+10	eval-logloss:-9.4776e+09
[282]	train-logloss:-1.05369e+10	eval-logloss:-9.4776e+09
[283]	train-logloss:-1.0537e+10	eval-logloss:-9.4776e+09
[284]	train-logloss:-1.0537e+10	eval-logloss:-9.4776e+09
[285]	train-logloss:-1.0537e+10	eval-logloss:-9.4776e+09
[286]	train-logloss:-1.05371e+10	eval-logloss:-9.47755e+09
[287]	train-logloss:-1.05371e+10	eval-logloss:-9.4783e+09
[288]	train-logloss:-1.05371e+10	eval-logloss:-9.47774e+09
[289]	train-logloss:-1.05371e+10	eval-logloss:-9.47774e+09
[290]	train-logloss:-1.05371e+10	eval-logloss:-9.47779e+09
[291]	train-logloss:-1.05371e+10	eval-logloss:-9.47835e+09
[292]	train-logloss:-1.05371e+10	eval-logloss:-9.47835e+09
[293]	train-logloss:-1.05372e+10	eval-logloss:-9.47835e+09
[294]	train-logloss:-1.05372e+10	eval-logloss:-9.47835e+09
[295]	train-logloss:-1.05372e+10	eval-logloss:-9.47832e+09
[296]	tra

[419]	train-logloss:-1.05387e+10	eval-logloss:-9.47841e+09
[420]	train-logloss:-1.05387e+10	eval-logloss:-9.47841e+09
[421]	train-logloss:-1.05387e+10	eval-logloss:-9.47835e+09
[422]	train-logloss:-1.05387e+10	eval-logloss:-9.47835e+09
[423]	train-logloss:-1.05387e+10	eval-logloss:-9.47835e+09
[424]	train-logloss:-1.05387e+10	eval-logloss:-9.47835e+09
[425]	train-logloss:-1.05387e+10	eval-logloss:-9.47836e+09
[426]	train-logloss:-1.05387e+10	eval-logloss:-9.47836e+09
[427]	train-logloss:-1.05387e+10	eval-logloss:-9.47836e+09
[428]	train-logloss:-1.05388e+10	eval-logloss:-9.47835e+09
[429]	train-logloss:-1.05388e+10	eval-logloss:-9.47836e+09
[430]	train-logloss:-1.05388e+10	eval-logloss:-9.47836e+09
[431]	train-logloss:-1.05388e+10	eval-logloss:-9.47835e+09
[432]	train-logloss:-1.05388e+10	eval-logloss:-9.47835e+09
[433]	train-logloss:-1.05388e+10	eval-logloss:-9.47835e+09
[434]	train-logloss:-1.05388e+10	eval-logloss:-9.47835e+09
[435]	train-logloss:-1.05388e+10	eval-logloss:-9.47835e+

[113]	train-logloss:-5.22648e+09	eval-logloss:-4.46056e+09
[114]	train-logloss:-5.22657e+09	eval-logloss:-4.46063e+09
[115]	train-logloss:-5.22662e+09	eval-logloss:-4.46062e+09
[116]	train-logloss:-5.2267e+09	eval-logloss:-4.46063e+09
[117]	train-logloss:-5.2274e+09	eval-logloss:-4.46079e+09
[118]	train-logloss:-5.22752e+09	eval-logloss:-4.46083e+09
[119]	train-logloss:-5.22758e+09	eval-logloss:-4.46083e+09
[120]	train-logloss:-5.22767e+09	eval-logloss:-4.45612e+09
[121]	train-logloss:-5.22774e+09	eval-logloss:-4.45618e+09
[122]	train-logloss:-5.22785e+09	eval-logloss:-4.45621e+09
[123]	train-logloss:-5.22795e+09	eval-logloss:-4.46085e+09
[124]	train-logloss:-5.22801e+09	eval-logloss:-4.45617e+09
[125]	train-logloss:-5.22814e+09	eval-logloss:-4.45617e+09
[126]	train-logloss:-5.22819e+09	eval-logloss:-4.45617e+09
[127]	train-logloss:-5.22867e+09	eval-logloss:-4.45606e+09
[128]	train-logloss:-5.22881e+09	eval-logloss:-4.46073e+09
[129]	train-logloss:-5.22886e+09	eval-logloss:-4.46073e+09

[253]	train-logloss:-5.23713e+09	eval-logloss:-4.46041e+09
[254]	train-logloss:-5.23714e+09	eval-logloss:-4.46041e+09
[255]	train-logloss:-5.23715e+09	eval-logloss:-4.4604e+09
[256]	train-logloss:-5.23716e+09	eval-logloss:-4.4604e+09
[257]	train-logloss:-5.23716e+09	eval-logloss:-4.46035e+09
[258]	train-logloss:-5.23717e+09	eval-logloss:-4.46035e+09
[259]	train-logloss:-5.2372e+09	eval-logloss:-4.4604e+09
[260]	train-logloss:-5.2372e+09	eval-logloss:-4.4604e+09
[261]	train-logloss:-5.23723e+09	eval-logloss:-4.46049e+09
[262]	train-logloss:-5.23724e+09	eval-logloss:-4.46059e+09
[263]	train-logloss:-5.23724e+09	eval-logloss:-4.46049e+09
[264]	train-logloss:-5.23723e+09	eval-logloss:-4.46059e+09
[265]	train-logloss:-5.23724e+09	eval-logloss:-4.46059e+09
[266]	train-logloss:-5.23727e+09	eval-logloss:-4.46056e+09
[267]	train-logloss:-5.2373e+09	eval-logloss:-4.46056e+09
[268]	train-logloss:-5.23731e+09	eval-logloss:-4.46056e+09
[269]	train-logloss:-5.23733e+09	eval-logloss:-4.46056e+09
[270

[393]	train-logloss:-5.23778e+09	eval-logloss:-4.46006e+09
[394]	train-logloss:-5.23778e+09	eval-logloss:-4.46006e+09
[395]	train-logloss:-5.23778e+09	eval-logloss:-4.46006e+09
[396]	train-logloss:-5.23778e+09	eval-logloss:-4.46006e+09
[397]	train-logloss:-5.23778e+09	eval-logloss:-4.46006e+09
[398]	train-logloss:-5.23778e+09	eval-logloss:-4.46006e+09
[399]	train-logloss:-5.23778e+09	eval-logloss:-4.46006e+09
[400]	train-logloss:-5.23778e+09	eval-logloss:-4.46006e+09
[401]	train-logloss:-5.23778e+09	eval-logloss:-4.46006e+09
[402]	train-logloss:-5.23778e+09	eval-logloss:-4.46006e+09
[403]	train-logloss:-5.23778e+09	eval-logloss:-4.46006e+09
[404]	train-logloss:-5.23778e+09	eval-logloss:-4.46006e+09
[405]	train-logloss:-5.23778e+09	eval-logloss:-4.46006e+09
[406]	train-logloss:-5.23778e+09	eval-logloss:-4.46006e+09
[407]	train-logloss:-5.23778e+09	eval-logloss:-4.46006e+09
[408]	train-logloss:-5.23778e+09	eval-logloss:-4.46006e+09
[409]	train-logloss:-5.23778e+09	eval-logloss:-4.46006e+

In [496]:
def predict_rmsle(pred,y):
    squared_error = (np.log((y+1)/(pred+1)))** 2
    rmsle = np.sqrt(np.mean(squared_error))
    if math.isnan(rmsle):
        print("this is a nan")

    return rmsle

print(predict_rmsle(home_predicted, test_home_Y))
print(predict_rmsle(out_predicted, test_out_Y))

1.7424763915497483
1.594352490374371


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\series.py:679: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [ ]:
# 사이킷런 XGBoost를 통해 예측

param_grid = {"max_depth":[3,5], "eta":[0.1,0.05],"eval_metric":"logloss"}
num_rounds = 400

from xgboost import XGBClassifier
# 하이퍼 파라미터와 early stopping 파라미터를 train()함수의 파라미터로 전달
xgb_home_model = XGBClassifier().fit(home_X, home_Y)
xgb_out_model = XGBClassifier().fit(out_X, out_Y)
# eval 파라미터에 학습 데이터 셋과 eval 데이터 셋을 명시해주면 eval 데이터셋에 대해
# 평가를 수행하면서 조기 중단 적용 가능

gs_home = GridSearchCV(estimator=xgb_home_model, param_grid=param_grid,
                   scoring=rmsle,cv=3, n_jobs=-1)
gs_out = GridSearchCV(estimator=xgb_out_model, param_grid=param_grid,
                   scoring=rmsle,cv=3, n_jobs=-1)


In [ ]:
# 사이킷런 XGBoost를 통해 예측

param_grid = {"max_depth":[3,5], "eta":[0.1,0.05],"eval_metric":"logloss"}
num_rounds = 400

from xgboost import XGBClassifier
# 하이퍼 파라미터와 early stopping 파라미터를 train()함수의 파라미터로 전달
xgb_home_model = XGBClassifier().fit(home_X, home_Y)
xgb_out_model = XGBClassifier().fit(out_X, out_Y)
# eval 파라미터에 학습 데이터 셋과 eval 데이터 셋을 명시해주면 eval 데이터셋에 대해
# 평가를 수행하면서 조기 중단 적용 가능

gs_home = GridSearchCV(estimator=xgb_home_model, param_grid=param_grid,
                   scoring=rmsle,cv=3, n_jobs=-1)
gs_out = GridSearchCV(estimator=xgb_out_model, param_grid=param_grid,
                   scoring=rmsle,cv=3, n_jobs=-1)


In [ ]:
# 사이킷런 LightGBM을 통해 예측

param_grid = {"max_depth":[3,5,9], "eta":[0.1,0.05,0.01],"eval_metric":"logloss"}
num_rounds = 400

from lightgbm import LGBMClassifier
# 하이퍼 파라미터와 early stopping 파라미터를 train()함수의 파라미터로 전달
lgbm_home_model = LGBMClassifier().fit(home_X, home_Y)
lgbm_out_model = LGBMClassifier().fit(out_X, out_Y)
# eval 파라미터에 학습 데이터 셋과 eval 데이터 셋을 명시해주면 eval 데이터셋에 대해
# 평가를 수행하면서 조기 중단 적용 가능

gs_home = GridSearchCV(estimator=lgbm_home_model, param_grid=param_grid,
                   scoring=rmsle,cv=5, n_jobs=-1)
gs_out = GridSearchCV(estimator=lgbm_out_model, param_grid=param_grid,
                   scoring=rmsle,cv=5, n_jobs=-1)


In [ ]:
param_grid = {"max_depth":5, "eta":0.1,"eval_metric":"logloss"}
num_rounds = 400

# 하이퍼 파라미터와 early stopping 파라미터를 train()함수의 파라미터로 전달
xgb_home_model = xgb.train(dtrain=dhome, num_boost_round=num_rounds, params=param_grid,
                     early_stopping_rounds=400, evals=wlist_home)

xgb_out_model = xgb.train(dtrain=dhome, num_boost_round=num_rounds, params=param_grid,
                     early_stopping_rounds=400, evals=wlist_out)

# eval 파라미터에 학습 데이터 셋과 eval 데이터 셋을 명시해주면 eval 데이터셋에 대해
# 평가를 수행하면서 조기 중단 적용 가능


In [376]:
# 예측
home_predicted = xgb_home_model.predict(dhome)
out_predicted = xgb_out_model.predict(dout)

In [378]:
home_X

,CARD_SIDO_NM,STD_CLSS_NM,AGE,year_x,FLC,month,AMT_x,out_home_x,diff_x,검색량(=100)_x,confirmed_x,소비자심리지수_x
0,0,0,0,2020,0,2,360000,1,-0.106782,84.25142,7.0,97
1,0,0,0,2020,1,2,441100,1,-0.106782,84.25142,7.0,97
2,0,0,0,2020,1,3,427510,1,-0.106782,100.00000,29.0,79
3,0,0,0,2020,1,4,478500,1,-0.106782,40.21097,17.0,76
4,0,0,1,2020,0,2,75486461,1,-0.106782,84.25142,7.0,97
...,...,...,...,...,...,...,...,...,...,...,...,...
6161,16,40,0,2020,1,2,785500,0,-0.500314,84.25142,10.0,94
6162,16,40,0,2020,1,4,258800,0,-0.500314,40.21097,1.0,72
6163,16,40,1,2020,0,2,9217300,0,-0.500314,84.25142,10.0,94
6164,16,40,1,2020,0,3,1724650,0,-0.500314,100.00000,34.0,79


In [379]:
home_Y

0        -143800
1        1075900
2         203490
3        -212500
4       68946910
          ...   
6161      -17350
6162       21600
6163    16414944
6164    20152200
6165    10870750
Name: corona_effect, Length: 6166, dtype: int64

In [377]:
home_predicted

array([ 2532638.8 ,  3796359.2 ,  -231596.55, ..., 17954914.  ,
       23214770.  ,  7224224.5 ], dtype=float32)

In [ ]:
# XGB 부스터에 맞게 조정
# Dmatrix : numpy 입력 파라미터를 받아서 만들어지는 XGBoost만의 전용 데이터셋
# pandas의 데이터셋이라면 numpy의 데이터셋으로 변경해주는 것이 필요함
dhome = xgb.DMatrix(data=home_X, label=home_Y)
dout = xgb.DMatrix(data=out_X, label=out_Y)

param_grid = {"max_depth":[3,5,7], "eta":[0.1,0.01],"eval_metric":"logloss"}
num_rounds = 400

wlist = [(dtrain,"train"),(dtest,"eval")]

# 하이퍼 파라미터와 early stopping 파라미터를 train()함수의 파라미터로 전달
"""
xgb_home_model = xgb.train(dtrain=dhome, num_boost_round=num_rounds,
                     early_stopping_rounds=100, evals=wlist)

xgb_home_model = xgb.train(dtrain=dhome, num_boost_round=num_rounds,
                     early_stopping_rounds=100, evals=wlist)
"""
xgb_home = xgb()
xgb_out = xgb()

# eval 파라미터에 학습 데이터 셋과 eval 데이터 셋을 명시해주면 eval 데이터셋에 대해
# 평가를 수행하면서 조기 중단 적용 가능

# rmsle_scorer = metrics.make_scorer(rmsle, greater_is_better=False)
pipe_xgb = Pipeline([('scaler', StandardScaler()), ('reg', XGBClassifier())])
home_
"""
param_range = [0.01, 0.1, 1.0, 10.0]
param_grid = [{'scaler': [StandardScaler()],'reg__C': param_range, 'reg__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],'reg__gamma':['auto','scale']},
    {'scaler': [MinMaxScaler()],'reg__C': param_range, 'reg__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],'reg__gamma':['auto','scale']}]

"""
gs_home = GridSearchCV(estimator=xgb_home, param_grid=param_grid,
                   scoring=rmsle,cv=5, n_jobs=-1)
gs_out = GridSearchCV(estimator=xgb_out, param_grid=param_grid,
                   scoring=rmsle,cv=5, n_jobs=-1)

gs_home.fit(home_features, home_target)
gs_out.fit(out_features, out_target)

In [341]:
home_train.iloc[:,:12]

,CARD_SIDO_NM,STD_CLSS_NM,AGE,year_x,FLC,month,AMT_x,out_home_x,diff_x,검색량(=100)_x,confirmed_x,소비자심리지수_x
0,강원,건강보조식품 소매업,0,2020,0,2,360000,1,-0.106782,84.25142,7.0,97
1,강원,건강보조식품 소매업,0,2020,1,2,441100,1,-0.106782,84.25142,7.0,97
2,강원,건강보조식품 소매업,0,2020,1,3,427510,1,-0.106782,100.00000,29.0,79
3,강원,건강보조식품 소매업,0,2020,1,4,478500,1,-0.106782,40.21097,17.0,76
4,강원,건강보조식품 소매업,1,2020,0,2,75486461,1,-0.106782,84.25142,7.0,97
...,...,...,...,...,...,...,...,...,...,...,...,...
6161,충북,휴양콘도 운영업,0,2020,1,2,785500,0,-0.500314,84.25142,10.0,94
6162,충북,휴양콘도 운영업,0,2020,1,4,258800,0,-0.500314,40.21097,1.0,72
6163,충북,휴양콘도 운영업,1,2020,0,2,9217300,0,-0.500314,84.25142,10.0,94
6164,충북,휴양콘도 운영업,1,2020,0,3,1724650,0,-0.500314,100.00000,34.0,79


In [342]:
home_train

,CARD_SIDO_NM,STD_CLSS_NM,AGE,year_x,FLC,month,AMT_x,out_home_x,diff_x,검색량(=100)_x,confirmed_x,소비자심리지수_x,year_y,AMT_y,out_home_y,diff_y,검색량(=100)_y,confirmed_y,소비자심리지수_y,corona_effect
0,강원,건강보조식품 소매업,0,2020,0,2,360000,1,-0.106782,84.25142,7.0,97,2019,216200,1,-0.106782,0.02146,0.0,97,-143800
1,강원,건강보조식품 소매업,0,2020,1,2,441100,1,-0.106782,84.25142,7.0,97,2019,1517000,1,-0.106782,0.02146,0.0,97,1075900
2,강원,건강보조식품 소매업,0,2020,1,3,427510,1,-0.106782,100.00000,29.0,79,2019,631000,1,-0.106782,0.02302,0.0,96,203490
3,강원,건강보조식품 소매업,0,2020,1,4,478500,1,-0.106782,40.21097,17.0,76,2019,266000,1,-0.106782,0.02953,0.0,98,-212500
4,강원,건강보조식품 소매업,1,2020,0,2,75486461,1,-0.106782,84.25142,7.0,97,2019,144433371,1,-0.106782,0.02146,0.0,97,68946910
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6161,충북,휴양콘도 운영업,0,2020,1,2,785500,0,-0.500314,84.25142,10.0,94,2019,768150,0,-0.500314,0.02146,0.0,98,-17350
6162,충북,휴양콘도 운영업,0,2020,1,4,258800,0,-0.500314,40.21097,1.0,72,2019,280400,0,-0.500314,0.02953,0.0,99,21600
6163,충북,휴양콘도 운영업,1,2020,0,2,9217300,0,-0.500314,84.25142,10.0,94,2019,25632244,0,-0.500314,0.02146,0.0,98,16414944
6164,충북,휴양콘도 운영업,1,2020,0,3,1724650,0,-0.500314,100.00000,34.0,79,2019,21876850,0,-0.500314,0.02302,0.0,99,20152200


In [325]:
before

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,year,FLC,month,AMT,out_home,diff,검색량(=100),confirmed,소비자심리지수
0,강원,건강보조식품 소매업,강원,0,2019,0,2,216200,1,-0.106782,0.02146,0.0,97
1,강원,건강보조식품 소매업,강원,0,2019,1,2,1517000,1,-0.106782,0.02146,0.0,97
2,강원,건강보조식품 소매업,강원,1,2019,0,2,144433371,1,-0.106782,0.02146,0.0,97
3,강원,골프장 운영업,강원,0,2019,0,2,1629420,1,0.034461,0.02146,0.0,97
4,강원,골프장 운영업,강원,0,2019,1,2,8254950,1,0.034461,0.02146,0.0,97
...,...,...,...,...,...,...,...,...,...,...,...,...,...
50674,전북,정기 항공 운송업,충북,1,2019,1,4,841500,1,-0.862732,0.02953,0.0,99
50675,제주,정기 항공 운송업,충북,0,2019,0,4,2951200,1,-0.862732,0.02953,0.0,99
50676,제주,정기 항공 운송업,충북,0,2019,1,4,9762300,1,-0.862732,0.02953,0.0,99
50677,제주,정기 항공 운송업,충북,1,2019,0,4,42698507,1,-0.862732,0.02953,0.0,99


In [ ]:
home

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn import metrics
import math

# rmsle_scorer = metrics.make_scorer(rmsle, greater_is_better=False)
pipe_svr = Pipeline([('scaler', StandardScaler()), ('reg', SVR())])
"""
param_range = [0.01, 0.1, 1.0, 10.0]
param_grid = [{'scaler': [StandardScaler()],'reg__C': param_range, 'reg__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],'reg__gamma':['auto','scale']},
    {'scaler': [MinMaxScaler()],'reg__C': param_range, 'reg__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],'reg__gamma':['auto','scale']}]

"""
gs_home = GridSearchCV(estimator=pipe_svr, param_grid=param_grid,
                   scoring=rmsle,cv=5, n_jobs=-1)
gs_out = GridSearchCV(estimator=pipe_svr, param_grid=param_grid,
                   scoring=rmsle,cv=5, n_jobs=-1)
gs_home.fit(home_features, home_target)
gs_out.fit(out_features, out_target)

In [305]:
xgb_wrapper = XGBClassifier(n_estimaotrs=400, learning_rate = 0.1, max_depth=3)
xgb_wrapper.fit(X_train, y_train, early_stopping_rounds=100)
w_preds = xgb_wrapper.predict(X_test)
w_pred_proba = xgb_wrapper.predict_proba(X_test)[:,1]

# early stooping roudns -> 100번의 반복동안 더이상 성능 평가 지수가 향상되지 않는다면
# 반복을 멈춤

# 만약 100보다 더 적은 숫자로 하면 성능 저하가 많이 일어날 가능성이 o

,CARD_SIDO_NM,STD_CLSS_NM,year,month,AMT,out_home,diff,검색량(=100),confirmed,소비자심리지수
0,강원,건강보조식품 소매업,2019,2,146715871,4,-0.713958,0.08584,0.0,389
1,강원,건강보조식품 소매업,2019,3,109439025,4,-0.427127,0.09208,0.0,384
2,강원,건강보조식품 소매업,2019,4,106548473,7,-1.894795,0.20671,0.0,689
3,강원,골프장 운영업,2019,2,1152240972,42,0.996796,0.90132,0.0,4102
4,강원,골프장 운영업,2019,3,2760497139,21,0.510255,0.48342,0.0,2052
...,...,...,...,...,...,...,...,...,...,...
1885,충북,화장품 및 방향제 소매업,2019,3,729328422,0,-4.467664,0.46040,0.0,1981
1886,충북,화장품 및 방향제 소매업,2019,4,687254347,0,-4.667453,0.62013,0.0,2102
1887,충북,휴양콘도 운영업,2019,2,165872181,0,-10.646937,0.40774,0.0,1851
1888,충북,휴양콘도 운영업,2019,3,107431443,0,-10.646937,0.43738,0.0,1856


In [306]:
after_result

,CARD_SIDO_NM,STD_CLSS_NM,year,month,AMT,out_home,diff,검색량(=100),confirmed,소비자심리지수
0,강원,건강보조식품 소매업,2020,2,76505561,4,-0.713958,337.00568,28.0,385
1,강원,건강보조식품 소매업,2020,3,96059012,3,-0.607176,300.00000,87.0,238
2,강원,건강보조식품 소매업,2020,4,88823988,4,-1.000788,160.84388,68.0,298
3,강원,골프장 운영업,2020,2,968456455,28,0.680340,2359.03976,196.0,2674
4,강원,골프장 운영업,2020,3,2915797995,12,0.294962,1200.00000,348.0,947
...,...,...,...,...,...,...,...,...,...,...
1837,충북,화장품 및 방향제 소매업,2020,3,428881434,0,-3.268930,1400.00000,476.0,1110
1838,충북,화장품 및 방향제 소매업,2020,4,450507431,0,-2.469774,402.10970,10.0,738
1839,충북,휴양콘도 운영업,2020,2,47085830,0,-8.377217,1263.77130,150.0,1425
1840,충북,휴양콘도 운영업,2020,3,12733490,0,-3.837777,700.00000,238.0,557


In [299]:
after

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,year,FLC,month,AMT,out_home,diff,검색량(=100),confirmed,소비자심리지수
205707,강원,건강보조식품 소매업,강원,1020s,2020,family,2,360000,1,-0.106782,84.25142,7.0,97
205708,강원,건강보조식품 소매업,강원,1020s,2020,single,2,441100,1,-0.106782,84.25142,7.0,97
205709,강원,건강보조식품 소매업,강원,3060s,2020,family,2,75486461,1,-0.106782,84.25142,7.0,97
205710,강원,골프장 운영업,강원,1020s,2020,family,2,362300,1,0.034461,84.25142,7.0,97
205711,강원,골프장 운영업,강원,1020s,2020,single,2,2922335,1,0.034461,84.25142,7.0,97
...,...,...,...,...,...,...,...,...,...,...,...,...,...
265430,서울,정기 항공 운송업,충북,1020s,2020,single,4,3017380,0,-0.862732,40.21097,183.0,72
265431,서울,정기 항공 운송업,충북,3060s,2020,family,4,37190900,0,-0.862732,40.21097,183.0,72
265432,인천,정기 항공 운송업,충북,1020s,2020,single,4,54700,1,-0.862732,40.21097,29.0,72
265433,제주,정기 항공 운송업,충북,1020s,2020,single,4,480500,1,-0.862732,40.21097,4.0,72
